In [51]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [52]:
from Utils.recsys2022DataReader import createBumpURM
from Utils.recsys2022DataReader import createSmallICM
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Hybrid.LinearHybridRecommender import LinearHybridTwoRecommender
from optuna.samplers import TPESampler
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
import optuna as op

In [53]:
URM = createBumpURM()
ICM = createSmallICM()

C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Utils\recsys2022DataReader.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(urmPath)


In [54]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, 0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 918 ( 2.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1564 ( 3.8%) Users that have less than 1 test interactions


In [55]:
ItemKNNCF_recommender = ItemKNNCFRecommender(URM_train)
ItemKNNCBF_recommender = ItemKNNCBFRecommender(URM_train, ICM)

ItemKNNCBFRecommender: ICM Detected 4877 (19.9%) items with no features.


In [58]:
def objective(trial):

    topKCF = trial.suggest_int("topKCF", 300, 400)
    shrinkCF = trial.suggest_float("shrinkCF", 10, 100)
    topKCBF = trial.suggest_int("topKCBF", 300, 400)
    shrinkCBF = trial.suggest_float("shrinkCBF", 10, 100)
    alpha = trial.suggest_float("alpha", 0.1, 0.9)

    ItemKNNCF_recommender.fit(topK=topKCF, shrink=shrinkCF)
    ItemKNNCBF_recommender.fit(topK=topKCBF, shrink=shrinkCBF)

    recommender = LinearHybridTwoRecommender(URM_train, Recommender_1=ItemKNNCF_recommender, Recommender_2=ItemKNNCBF_recommender)
    recommender.fit(alpha=alpha)

    result, _ = evaluator_validation.evaluateRecommender(recommender)

    return result.loc[10]['MAP']

In [59]:
study = op.create_study(direction='maximize', sampler=TPESampler(multivariate=True))
study.optimize(objective, n_trials=1)

C:\Users\Mike\anaconda3\envs\recsys\lib\site-packages\optuna\samplers\_tpe\sampler.py:263: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2022-11-11 16:03:15,761] A new study created in memory with name: no-name-99b2d761-9fea-45bd-99e1-bd36318b289e


Similarity column 24507 (100.0%), 6114.82 column/sec. Elapsed time 4.01 sec
Compute_Similarity: detected dense matrix
Similarity column 24507 (100.0%), 1475.11 column/sec. Elapsed time 16.61 sec
EvaluatorHoldout: Processed 40065 (100.0%) in 41.94 sec. Users per second: 955


[I 2022-11-11 16:04:18,909] Trial 0 finished with value: 0.016423049647497304 and parameters: {'topKCF': 354, 'shrinkCF': 84.5322249059575, 'topKCBF': 390, 'shrinkCBF': 80.88989157475186, 'alpha': 0.8508702365392337}. Best is trial 0 with value: 0.016423049647497304.


In [ ]:
ItemKNNCF_recommender.fit(topK=study.best_params['topKCF'], shrink=study.best_params['shrinkCF'])
ItemKNNCBF_recommender.fit(topK=study.best_params['topKCBF'], shrink=study.best_params['shrinkCBF'])

recommender = LinearHybridTwoRecommender(URM_train, Recommender_1=ItemKNNCF_recommender, Recommender_2=ItemKNNCBF_recommender, verbose=False)
recommender.fit(alpha=study.best_params['alpha'])

In [ ]:
result, _ = evaluator_test.evaluateRecommender(recommender)
result